## Using a Quantum Computer to Train XOR

First we import the dwave library

In [4]:
import dwavebinarycsp
import dwavebinarycsp.factories.constraint.gates as gates
import operator

### Let's take a look at what we have

In [21]:
!dwave solvers

Solver: DW_2000Q_3
  Parameters:
    anneal_offsets: A list of anneal offsets for each working qubit (NaN if u...
    anneal_schedule: A piecewise linear annealing schedule specified by a list...
    annealing_time: A positive integer that sets the duration (in microsecond...
    answer_mode: ?
    auto_scale: ?
    beta: Boltzmann distribution parameter. Only used when postproc...
    chains: Defines which qubits represent the same logical variable ...
    flux_biases: A list of flux biases for each working qubit.
    flux_drift_compensation: A boolean for whether to activate the flux drift compensa...
    initial_state: The initial states to use for the reverse anneal feature.
    max_answers: ?
    num_reads: ?
    num_spin_reversal_transforms: ?
    postprocess: either 'sampling' or 'optimization'
    programming_thermalization: An integer that gives the time (in microseconds) to wait ...
    readout_thermalization: An integer that gives the time (in microseconds) to wait ...
    r

The solver we are using is DWave's 2000 Qubit Quantum Annealer.

Let's run a sample problem on their quantum computer to check if it is up and to see their response times.

But before that, let's first ping their Quantum Computer and see what we get

In [22]:
!dwave ping
!dwave sample --random-problem

Using endpoint: https://cloud.dwavesys.com/sapi
Using solver: DW_2000Q_3
Submitted problem ID: 5b035fe5-a35c-4792-a4ff-b1a74b342e71

Wall clock time:
 * Solver definition fetch: 5313.781 ms
 * Problem submit and results fetch: 3958.114 ms
 * Total: 9271.895 ms

QPU timing:
 * total_real_time = 9346 us
 * qpu_access_overhead_time = 1044 us
 * anneal_time_per_run = 20 us
 * post_processing_overhead_time = 239 us
 * qpu_sampling_time = 164 us
 * readout_time_per_run = 123 us
 * qpu_delay_time_per_sample = 21 us
 * qpu_anneal_time_per_sample = 20 us
 * total_post_processing_time = 239 us
 * qpu_programming_time = 9182 us
 * run_time_chip = 164 us
 * qpu_access_time = 9346 us
 * qpu_readout_time_per_sample = 123 us
Using endpoint: https://cloud.dwavesys.com/sapi
Using solver: DW_2000Q_3
Using qubit biases: {0: -1.9623957726258783, 2: -0.3636483789484126, 3: 0.4914949866740108, 4: -1...
Using qubit couplings: {(1634, 1638): 0.4283406131909504, (587, 590): 0.7831580874393123, (642, 6...
Numbe

Here we can again see that we are using their 2000 QuBit Quantum Computer.
Furthermore, we see that it takes only **9 milliseconds** to send a request and get the result back.

That is **SUPER FAST**!!!

### Implementing a XOR gate using AND and NOT gates

In [5]:
def xor(a, b, c):
    not1= not a
    not2= not b
    and1= b and not1
    and2= not2 and a
    or1= and1 or and2
    return (c==or1)

Adding the constraints under which the quantum annealing process will happen.

Here, 'a' and 'b' are our input variables and 'c' is our output variable

In [6]:
csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)
csp.add_constraint(xor, ['a', 'b', 'c'])

bqm = dwavebinarycsp.stitch(csp)

We will run the same problem for 1000 iterations to make sure that we get a large sample to choose the correct answers from

In [7]:
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

# Set up a D-Wave system as the sampler
sampler = EmbeddingComposite(DWaveSampler())
import time
start_time=time.time()
#Running the process for 1000 Iterations
response = sampler.sample(bqm, num_reads=1000)
request_time=time.time()-start_time
print(f"Total Time Taken to get result: {round(request_time, 2)} sec")

valid, invalid, data = 0, 0, []
for datum in response.data(['sample', 'energy', 'num_occurrences']):
    if (csp.check(datum.sample)):
        valid = valid+datum.num_occurrences
        for i in range(datum.num_occurrences):
            data.append((datum.sample, datum.energy, '1'))
    else:
        invalid = invalid+datum.num_occurrences
        for i in range(datum.num_occurrences):
            data.append((datum.sample, datum.energy, '0'))
print(f"\nNo. of Low Energy state answers: {valid}")
print(f"No. of High Energy State Answers: {invalid}")

Total Time Taken to get result: 2.73 sec

No. of Low Energy state answers: 998
No. of High Energy State Answers: 2


### Let's look at some of the answers and their Energy States

In [12]:
results_dict={}
total_time=0
qpu_time=0

for sample, energy in response.data(['sample', 'energy']):
    
    a_value=sample['a']
    b_value=sample['b']
    c_value=sample['c']
    
    if (a_value, b_value, c_value) not in results_dict:
        results_dict[(a_value, b_value, c_value)] = energy
    
    total_time=total_time+response.info['timing']['total_real_time']
    qpu_time=qpu_time+response.info['timing']['qpu_anneal_time_per_sample']

for result in results_dict:
    print(f"a, b, c = {result} \nEnergy = {results_dict[result]} \n______")

print(f"\nTotal Time Required by Quantum Computer: {total_time/1000000} seconds")
print(f"Average Time Required in Quantum Computer per Iteration: {qpu_time/6} microseconds!")

a, b, c = (1, 1, 0) 
Energy = 0.0 
______
a, b, c = (1, 0, 1) 
Energy = 0.0 
______
a, b, c = (0, 1, 1) 
Energy = 0.0 
______
a, b, c = (0, 0, 0) 
Energy = 0.0 
______
a, b, c = (1, 1, 1) 
Energy = 2.0 
______

Total Time Required by Quantum Computer: 1.039032 seconds
Average Time Required in Quantum Computer per Iteration: 20.0 microseconds!


### To Recap

We ran the XOR gate training for 1000 Iterations on a Quantum Computer to get the best results
The total time taken to send a request and get the result back was 2.73 seconds.

If we strip away the time that was taken to send and receive a request, then the total time spent in the quantum computer was 1.04 seconds

And out of that, on average, each iteration took just 20 microseconds to run! That means all the 1000 iterations was completed in 20 milliseconds and each iteration took 20 microseconds

The rest of the time was taken by DWave to process the request, place the request in the quantum computer, read the output and preprocess it. These took apporximately:

qpu_access_overhead_time: 2103,
post_processing_overhead_time: 4137,
qpu_sampling_time: 164000,
readout_time_per_run: 123,
qpu_delay_time_per_sample: 21,
qpu_anneal_time_per_sample: 20,
total_post_processing_time: 4137,
qpu_programming_time: 9172,
run_time_chip: 164000,
qpu_access_time: 173172,
qpu_readout_time_per_sample: 123